<a href="https://colab.research.google.com/github/adriari4/lab-langsmith-summarizations/blob/main/lab-langsmith-sumarizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACE")

In [ ]:
from getpass import getpass
import os

# Ask for keys securely (input will not be shown)
OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
LANGCHAIN_API_KEY = getpass("Enter your LANGCHAIN_API_KEY: ")
HUGGINGFACEHUB_API_TOKEN = getpass("Enter your HUGGINGFACEHUB_API_TOKEN: ")

# Store them as environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

print("🔐 API keys loaded securely.")


Enter your OPENAI_API_KEY: ··········
Enter your LANGCHAIN_API_KEY: ··········
Enter your HUGGINGFACEHUB_API_TOKEN: ··········
🔐 API keys loaded securely.


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [ ]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [ ]:
!pip install datasets==2.19.1


In [ ]:
from datasets import load_dataset
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0")
print(cnn_dataset)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [ ]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
cnn_dataset['train'][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
from datasets import Dataset

MAX_NEWS=10

# Manually create a small sample of data for summarization evaluation
# This mimics the structure of the cnn_dailymail dataset to bypass the loading issue.
sample_data = [
    {
        'article': "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force \"to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction.\" It's a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he wants to. \"While I believe I have the authority to carry out this military action without specific congressional authorization, I know that the country will be stronger if we take this course, and our actions will be even more effective,\" he said. \"We should have this debate, because the issues are too big for business as usual.\" Obama said top congressional leaders had agreed to schedule a debate when the body returns to Washington on September 9. The Senate Foreign Relations Committee will hold a hearing over the matter on Tuesday, Sen. Robert Menendez said. Transcript: Read Obama's full remarks . Syrian crisis: Latest developments . U.N. inspectors leave Syria . Obama's remarks came shortly after U.N. inspectors left Syria, carrying evidence that will determine whether chemical weapons were used in an attack early last week in a Damascus suburb. \"The aim of the game here, the mandate, is very clear -- and that is to ascertain whether chemical weapons were used -- and not by whom,\" U.N. spokesman Martin Nesirky told reporters on Saturday. But who used the weapons in the reported toxic gas attack in a Damascus suburb on August 21 has been a key point of global debate over the Syrian crisis. Top U.S. officials have said there's no doubt that the Syrian government was behind it, while Syrian officials have denied responsibility and blamed jihadists fighting with the rebels. British and U.S. intelligence reports say the attack involved chemical weapons, but U.N. officials have stressed the importance of waiting for an official report from inspectors. The inspectors will share their findings with U.N. Secretary-General Ban Ki-moon Ban, who has said he wants to wait until the U.N. team's final report is completed before presenting it to the U.N. Security Council. The Organization for the Prohibition of Chemical Weapons, which nine of the inspectors belong to, said Saturday that it could take up to three weeks to analyze the evidence they collected. \"It needs time to be able to analyze the information and the samples,\" Nesirky said. He noted that Ban has repeatedly said there is no alternative to a political solution to the crisis in Syria, and that \"a military solution is not an option.\" Bergen:  Syria is a problem from hell for the U.S. Obama: 'This menace must be confronted' Obama's senior advisers have debated the next steps to take, and the president's comments Saturday came amid mounting political pressure over the situation in Syria. Some U.S. lawmakers have called for immediate action while others warn of stepping into what could become a quagmire. Some global leaders have expressed support, but the British Parliament's vote against military action earlier this week was a blow to Obama's hopes of getting strong backing from key NATO allies. On Saturday, Obama proposed what he said would be a limited military action against Syrian President Bashar al-Assad. Any military attack would not be open-ended or include U.S. ground forces, he said. Syria's alleged use of chemical weapons earlier this month \"is an assault on human dignity,\" the president said. A failure to respond with force, Obama argued,  \"could lead to escalating use of chemical weapons or their proliferation to terrorist groups who would do our people harm. In a world with many dangers, this menace must be confronted.\" Syria missile strike: What would happen next? Map: U.S. and allied assets around Syria . Obama decision came Friday night . On Friday night, the president made a last-minute decision to consult lawmakers. What will happen if they vote no? It's unclear. A senior administration official told CNN that Obama has the authority to act without Congress -- even if Congress rejects his request for authorization to use force. Obama on Saturday continued to shore up support for a strike on the al-Assad government. He spoke by phone with French President Francois Hollande before his Rose Garden speech. \"The two leaders agreed that the international community must deliver a resolute message to the Assad regime -- and others who would consider using chemical weapons -- that these crimes are unacceptable and those who violate this international norm will be held accountable by the world,\" the White House said. Meanwhile, as uncertainty loomed over how Congress would weigh in, U.S. military officials said they remained at the ready. 5 key assertions: U.S. intelligence report on Syria . Syria: Who wants what after chemical weapons horror . Reactions mixed to Obama's speech . A spokesman for the Syrian National Coalition said that the opposition group was disappointed by Obama's announcement. \"Our fear now is that the lack of action could embolden the regime and they repeat his attacks in a more serious way,\" said spokesman Louay Safi. \"So we are quite concerned.\" Some members of Congress applauded Obama's decision. House Speaker John Boehner, Majority Leader Eric Cantor, Majority Whip Kevin McCarthy and Conference Chair Cathy McMorris Rodgers issued a statement Saturday praising the president. \"Under the Constitution, the responsibility to declare war lies with Congress,\" the Republican lawmakers said. \"We are glad the president is seeking authorization for any military action in Syria in response to serious, substantive questions being raised.\" More than 160 legislators, including 63 of Obama's fellow Democrats, had signed letters calling for either a vote or at least a \"full debate\" before any U.S. action. British Prime Minister David Cameron, whose own attempt to get lawmakers in his country to support military action in Syria failed earlier this week, responded to Obama's speech in a Twitter post Saturday. \"I understand and support Barack Obama's position on Syria,\" Cameron said. An influential lawmaker in Russia -- which has stood by Syria and criticized the United States -- had his own theory. \"The main reason Obama is turning to the Congress:  the military operation did not get enough support either in the world, among allies of the US or in the United States itself,\" Alexei Pushkov, chairman of the international-affairs committee of the Russian State Duma, said in a Twitter post. In the United States, scattered groups of anti-war protesters around the country took to the streets Saturday. \"Like many other Americans...we're just tired of the United States getting involved and invading and bombing other countries,\" said Robin Rosecrans, who was among hundreds at a Los Angeles demonstration. What do Syria's neighbors think? Why Russia, China, Iran stand by Assad . Syria's government unfazed . After Obama's speech, a military and political analyst on Syrian state TV said Obama is \"embarrassed\" that Russia opposes military action against Syria, is \"crying for help\" for someone to come to his rescue and is facing two defeats -- on the political and military levels. Syria's prime minister appeared unfazed by the saber-rattling. \"The Syrian Army's status is on maximum readiness and fingers are on the trigger to confront all challenges,\" Wael Nader al-Halqi said during a meeting with a delegation of Syrian expatriates from Italy, according to a banner on Syria State TV that was broadcast prior to Obama's address. An anchor on Syrian state television said Obama \"appeared to be preparing for an aggression on Syria based on repeated lies.\" A top Syrian diplomat told the state television network that Obama was facing pressure to take military action from Israel, Turkey, some Arabs and right-wing extremists in the United States. \"I think he has done well by doing what Cameron did in terms of taking the issue to Parliament,\" said Bashar Jaafari, Syria's ambassador to the United Nations. Both Obama and Cameron, he said, \"climbed to the top of the tree and don't know how to get down.\" The Syrian government has denied that it used chemical weapons in the August 21 attack, saying that jihadists fighting with the rebels used them in an effort to turn global sentiments against it. British intelligence had put the number of people killed in the attack at more than 350. On Saturday, Obama said \"all told, well over 1,000 people were murdered.\" U.S. Secretary of State John Kerry on Friday cited a death toll of 1,429, more than 400 of them children. No explanation was offered for the discrepancy. Iran: U.S. military action in Syria would spark 'disaster' Opinion: Why strikes in Syria are a bad idea .",
        'highlights': "Syrian official: Obama climbed to the top of the tree, \"doesn't know how to get down\"\nObama sends a letter to the heads of the House and Senate .\nObama to seek congressional approval on military action against Syria .\nAim is to determine whether CW were used, not by whom, says U.N. spokesman .",
        'id': '0001d1afc246a7964130f43ae940af6bc6c57f01'
    },
    {
        'article': "(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's \"The Dukes of Hazzard,\" died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when \"The Dukes of Hazzard's\" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his \"hot pursuit\" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive \"kew-kew-kew\" chuckle and for goofy catchphrases such as \"cuff 'em and stuff 'em!\" upon making an arrest. Among the most popular shows on TV in the early '80s, \"The Dukes of Hazzard\" ran until 1985 and spawned TV movies, an animated series and video games. Several of Best's \"Hazzard\" co-stars paid tribute to the late actor on social media. \"I laughed and learned more from Jimmie in one hour than from anyone else in a whole year,\" co-star John Schneider, who played Bo Duke, said on Twitter. \"Give Uncle Jesse my love when you see him dear friend.\" \"Jimmy Best was the most constantly creative person I have ever known,\" said Ben Jones, who played mechanic Cooter on the show, in a Facebook post. \"Every minute of his long life was spent acting, writing, producing, painting, teaching, fishing, or involved in another of his life's many passions.\" Born Jewel Guy on July 26, 1926, in Powderly, Kentucky, Best was orphaned at 3 and adopted by Armen and Essa Best, who renamed him James and raised him in rural Indiana. Best served in the Army during World War II before launching his acting career. In the 1950s and 1960s, he accumulated scores of credits, playing a range of colorful supporting characters in such TV shows as \"The Twilight Zone,\" \"Bonanza,\" \"The Andy Griffith Show\" and \"Gunsmoke.\" He later appeared in a handful of Burt Reynolds' movies, including \"Hooper\" and \"The End.\" But Best will always be best known for his \"Hazzard\" role, which lives on in reruns. \"Jimmie was my teacher, mentor, close friend and collaborator for 26 years,\" Latshaw said. \"I directed two of his feature films, including the recent 'Return of the Killer Shrews,' a sequel he co-wrote and was quite proud of as he had made the first one more than 50 years earlier.\" People we've lost in 2015 . CNN's Stella Chan contributed to this story.",
        'highlights': "James Best, who played the sheriff on \"The Dukes of Hazzard,\" died Monday at 88 .\n\"Hazzard\" ran from 1979 to 1985 and was among the most popular shows on TV .",
        'id': '00200e794fa41d3f7ce92cbf43e9fd4cd652bb09'
    }
]

# Create a Dataset object from the list of dictionaries
cnn_dataset_synthetic = Dataset.from_list(sample_data)

# Apply the add_prefix function defined previously. Note: 'cnn_dataset' is no longer used here.
sample_cnn = cnn_dataset_synthetic.map(add_prefix)

sample_cnn

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [ ]:
print(sample_cnn[0])

{'article': 'Summarize this news:\nIt\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not becaus

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [ ]:
import datetime

In [ ]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [ ]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [ ]:
!pip install langchain-community
from langchain_community.llms import HuggingFaceHub

In [ ]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

/tmp/ipython-input-3938980585.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEndpoint``.
  summarizer_base = HuggingFaceHub(


In [ ]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [ ]:
!pip install -U langchain==1.1.0 langchain-core==1.1.0


In [ ]:
!pip install -U langchain langchain-core


In [ ]:
!pip install -U langsmith==0.5.6


ERROR: Ignored the following yanked versions: 0.0.57, 0.0.62, 0.1.65, 0.1.100, 0.1.130, 0.1.135, 0.1.136, 0.1.140, 0.1.141
ERROR: Could not find a version that satisfies the requirement langsmith==0.5.6 (from versions: 0.0.0rc0, 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.63, 0.0.64, 0.0.65, 0.0.66, 0.0.67, 0.0.68, 0.0.69, 0.0.70, 0.0.71, 0.0.72, 0.0.73, 0.0.74, 0.0.75, 0.0.76, 0.0.77, 0.0.78, 0.0.79, 0.0.80, 0.0.81, 0.0.82, 0.0.83, 0.0.84rc1, 0.0.84rc2, 0.0.84rc3, 0.0.84rc4, 0.0.84rc5, 0.0.84, 0.0.85, 0.0.86rc1, 0.0.86, 0.0.87, 0.0.88, 0.0.89, 0.0.90, 0.0.91, 0.0.92, 0.

In [ ]:
from langsmith import Client
from langsmith.evaluation import evaluate


In [ ]:
client = Client()

evaluation_config = {
    "evaluators": [
        "embedding_distance",
        # "string_distance"
    ]
}

In [ ]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

#evaluation_config = RunEvalConfig(
    #evaluators=[
        #"embedding_distance",
        #"string_distance"
    #],
#)



NameError: name 'RunEvalConfig' is not defined

### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [ ]:
from langsmith import Client
from langsmith.evaluation import evaluate
import datetime

client = Client()

# 1. Load dataset metadata
dataset = client.read_dataset(dataset_name=NAME_DATASET)

# 2. Fetch actual examples using dataset_id
examples = client.list_examples(dataset_id=dataset.id)





In [ ]:
# 1. Get the real Example objects from LangSmith
examples = list(client.list_examples(dataset_id=dataset.id))

# 2. Minimal evaluator (required so evaluate() does not crash)
def dummy_evaluator(example, prediction):
    return {"score": 1.0}

# 3. Run evaluation on LangSmith UI
results = evaluate(
    summarizer_base,           # target
    examples,                  # correct dataset type
    [dummy_evaluator],         # <---- FIXED
    client=client
)

results


View the evaluation results for experiment: 'complicated-hospital-12' at:
https://smith.langchain.com/o/691d53bb-01f5-4138-b552-cbf966a2cef9/datasets/319a6d0c-039f-415e-8103-c36fe1c77221/compare?selectedSessions=9d80758a-1c34-40f8-8899-c4846796ac65




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/llms.py", line 374, in invoke
    [self._convert_input(input)],
     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/llms.py", line 323, in _convert_input
    raise ValueError(msg)
ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
/usr/local/lib/python3.12/dist-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = u

,inputs.article,outputs.output,error,reference.highlights,feedback.dummy_evaluator,execution_time,example_id,id
0,Summarize this news:\nIt's official: U.S. Pres...,None,"ValueError(""Invalid input type <class 'dict'>....",Syrian official: Obama climbed to the top of t...,1.0,0.003680,8d167dcf-3703-4934-9019-6f9574a5a671,019abc16-d3c2-70bd-ae51-a88349770279
1,"Summarize this news:\n(CNN)James Best, best kn...",None,"ValueError(""Invalid input type <class 'dict'>....","James Best, who played the sheriff on ""The Duk...",1.0,0.001217,8e4e0253-0ce1-4345-a96b-29b427fea985,019abc16-d3cc-74d6-afbf-0ed58ffc608e


In [ ]:
def summarizer_wrapper(example, **kwargs):
    text = example.inputs["article"]
    summary = summarizer_base(text)
    return {"output": summary}


In [ ]:
from openai import OpenAI
import json

client_llm = OpenAI()

def llm_summary_evaluator(example, prediction):

    article = example.inputs["article"]
    reference = example.outputs["output"]
    generated = prediction.outputs["output"]

    prompt = f"""
You are an expert summarization evaluator.

Evaluate the quality of the GENERATED SUMMARY compared to the ARTICLE and REFERENCE SUMMARY.

Rate from 1 to 10 using:
- factual correctness
- faithfulness
- coverage of key points
- coherence
- conciseness

Return ONLY valid JSON:
{{
  "score": <number>,
  "reason": "<short explanation>"
}}

ARTICLE:
{article}

REFERENCE SUMMARY:
{reference}

GENERATED SUMMARY:
{generated}
"""

    response = client_llm.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    text = response.choices[0].message["content"]

    try:
        result = json.loads(text)
    except:
        result = {"score": 1.0, "reason": "Invalid JSON returned by evaluator LLM."}

    return result


In [ ]:
results = evaluate(
    summarizer_wrapper,
    examples,
    [llm_summary_evaluator],
    client=client
)

results


View the evaluation results for experiment: 'complicated-team-94' at:
https://smith.langchain.com/o/691d53bb-01f5-4138-b552-cbf966a2cef9/datasets/319a6d0c-039f-415e-8103-c36fe1c77221/compare?selectedSessions=797b1e02-8ae3-4e49-a26e-bf16935309ee




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: 'dict' object has no attribute 'inputs'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/tmp/ipython-input-3383131132.py", line 2, in summarizer_wrapper
    text = example.inputs["article"]
           ^^^^^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'inputs'
ERROR:langsmith.evaluation._runner:Error running evaluator <DynamicRunEvaluator llm_summary_evaluator> on run 019abc1b-cdf8-7714-87df-5fef32fe29aa: KeyError('article')
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1619, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(  # type: ignore[call-arg]
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/la

,inputs.article,outputs.output,error,reference.highlights,feedback.llm_summary_evaluator,execution_time,example_id,id
0,Summarize this news:\nIt's official: U.S. Pres...,None,"AttributeError(""'dict' object has no attribute...",Syrian official: Obama climbed to the top of t...,None,0.003286,8d167dcf-3703-4934-9019-6f9574a5a671,019abc1b-cdf8-7714-87df-5fef32fe29aa
1,"Summarize this news:\n(CNN)James Best, best kn...",None,"AttributeError(""'dict' object has no attribute...","James Best, who played the sheriff on ""The Duk...",None,0.002582,8e4e0253-0ce1-4345-a96b-29b427fea985,019abc1b-ce07-7062-a0f2-e2165dc672b5


In [ ]:
type(examples[0])


langsmith.schemas.Example

In [ ]:
%reset -f


In [ ]:
from langsmith import Client

# Crear el cliente con tu API key (ya cargada en el entorno)
client = Client()

# Cargar el dataset por nombre
dataset = client.read_dataset(dataset_name=NAME_DATASET)

# Cargar los Example objects reales
examples = list(client.list_examples(dataset_id=dataset.id))

print(type(examples[0]))        # debe mostrar langsmith.schemas.Example
print(examples[0].inputs)       # debe mostrar {"article": "..."}


NameError: name 'NAME_DATASET' is not defined

In [ ]:
def summarizer_wrapper(example):
    # example: langsmith.schemas.Example
    article = example.inputs["article"]

    return summarizer_base(article)


In [ ]:
!pip install langchain-openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
%reset -f

In [ ]:
from langsmith import Client

client = Client()

DATASET_ID = "319a6d0c-039f-415e-8103-c36fe1c77221"

# Cargar dataset por ID
dataset = client.read_dataset(dataset_id=DATASET_ID)

# Cargar los Example objects reales
examples = list(client.list_examples(dataset_id=DATASET_ID))

print("Type:", type(examples[0]))
print("Inputs:", examples[0].inputs)
print("Outputs:", examples[0].outputs)


Type: <class 'langsmith.schemas.Example'>
Inputs: {'article': 'Summarize this news:\nIt\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president s

In [ ]:
def get_inputs(example):
    """Supports both dict and Example objects."""
    if hasattr(example, "inputs"):
        return example.inputs
    return example["inputs"]

def get_outputs(example):
    """Supports both dict and Example objects."""
    if hasattr(example, "outputs"):
        return example.outputs
    return example["outputs"]


In [ ]:
def summarizer_wrapper(example):
    inputs = get_inputs(example)            # ← works for dict and Example
    article = inputs["article"]
    return summarizer_base(article)


In [ ]:
# 1. Import correcto
from langchain_openai import ChatOpenAI

# 2. Modelo LLM que actúa como evaluador
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. Helper functions (necesarias para dict + Example)
def get_inputs(example):
    if hasattr(example, "inputs"):
        return example.inputs
    return example["inputs"]

def get_outputs(example):
    if hasattr(example, "outputs"):
        return example.outputs
    return example["outputs"]

# 4. Evaluador LLM robusto
def llm_summary_evaluator(run, example):

    # predicted summary (LangSmith run object)
    predicted = run.outputs.get("output", "")

    # reference gold summary
    reference = get_outputs(example)["highlights"]

    # input article
    article = get_inputs(example)["article"]

    # evaluation prompt
    prompt = f"""
You are an expert summary evaluator. Rate from 1 to 10 the quality of the predicted summary compared to the reference.

ARTICLE:
{article}

REFERENCE SUMMARY:
{reference}

PREDICTED SUMMARY:
{predicted}

Respond ONLY with a number from 1 to 10.
"""

    score = llm.invoke(prompt).content.strip()
    return {"score": score}


In [ ]:
from langsmith.evaluation import evaluate


In [ ]:
results = evaluate(
    summarizer_wrapper,      # tu modelo wrapped
    examples,                # lista de Example objects
    [llm_summary_evaluator], # evaluador LLM
    client=client
)

results


View the evaluation results for experiment: 'aching-drawer-54' at:
https://smith.langchain.com/o/691d53bb-01f5-4138-b552-cbf966a2cef9/datasets/319a6d0c-039f-415e-8103-c36fe1c77221/compare?selectedSessions=79175495-6ff5-4190-85e2-775e914f541b




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: 'inputs'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/tmp/ipython-input-3580235278.py", line 2, in summarizer_wrapper
    inputs = get_inputs(example)            # ← works for dict and Example
             ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1901457283.py", line 11, in get_inputs
    return example["inputs"]
           ~~~~~~~^^^^^^^^^^
KeyError: 'inputs'
ERROR:langsmith.evaluation._runner:Error running evaluator <DynamicRunEvaluator llm_summary_evaluator> on run 019abc2a-e308-75bd-abde-155b6912812d: ValueError("Expected an EvaluationResult object, or dict with a metric 'key' and optional 'score'; got {'score': '1', 'key': 'llm_summary_evaluator'}")
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/evaluator.

,inputs.article,outputs.output,error,reference.highlights,feedback.llm_summary_evaluator,execution_time,example_id,id
0,Summarize this news:\nIt's official: U.S. Pres...,None,KeyError('inputs')\n\nTraceback (most recent c...,Syrian official: Obama climbed to the top of t...,None,0.003653,8d167dcf-3703-4934-9019-6f9574a5a671,019abc2a-e308-75bd-abde-155b6912812d
1,"Summarize this news:\n(CNN)James Best, best kn...",None,KeyError('inputs')\n\nTraceback (most recent c...,"James Best, who played the sheriff on ""The Duk...",None,0.003464,8e4e0253-0ce1-4345-a96b-29b427fea985,019abc2a-e7a8-7131-9451-1fed0eb4b602


In [1]:
# 3. Build evaluation data
#dataset_data = [
#    {"input": ex.inputs, "output": ex.outputs}
#    for ex in examples
#]

# 4. Run evaluation
#results = evaluate(
#    summarizer_base,               # target
#    dataset_data,                  # list of examples
#    ["embedding_distance"],        # evaluator
#    client=client
#)

#results

In [2]:
#project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

#base_t5_results = run_on_dataset(
    #client=client,
    #project_name=project_name,
    #dataset_name=NAME_DATASET,
    #llm_or_chain_factory=summarizer_base,
    #evaluation=evaluation_config,
#)

In [3]:
#Ignore the error shown below
#project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

#finetuned_t5_results = run_on_dataset(
#    client=client,
#    project_name=project_name,
#    dataset_name=NAME_DATASET,
 #   llm_or_chain_factory=summarizer_finetuned,
 #   evaluation=evaluation_config,
#)

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [ ]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [ ]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.